In [ ]:
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INSTALLATION GUIDE>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 

'''pip install nltk

pip install spacy==2.3.5

pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

pip install pyresparser'''

In [ ]:
# import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import *
import base64,random

In [ ]:
import os
import pandas as pd
from pyresparser import ResumeParser
import pdfplumber
import re

# Define the directory containing resumes
resumes_dir = 'cv_to_df/'
resumes_dir_full_path = os.path.abspath(resumes_dir)

def pdf_to_base64(file_path):
    
    encoded_pdf = base64.b64encode(file_path.read()).decode('utf-8')

    return encoded_pdf

def extract_text_from_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        pages = pdf.pages
        for page in pages:
            text += page.extract_text(x_tolerance=2)
    return text

# Create an empty DataFrame object to store the extracted data
df = pd.DataFrame()

# Iterate through all files in the resumes directory
for filename in os.listdir(resumes_dir):
    if filename.endswith('.pdf'):  # Check if the file is a PDF
        # Apply the ResumeParser function to the current file and append the extracted data to the DataFrame
        data = ResumeParser(os.path.join(resumes_dir, filename)).get_extracted_data()
        # Add a new column to the DataFrame containing the file path
        data['file_path'] = os.path.join(resumes_dir, filename)
        # Append the extracted data to the DataFrame
        df = df.append(data, ignore_index=True)

# Extract text from all PDFs in the resumes directory and create a new DataFrame
data = []
for filename in os.listdir(resumes_dir):
    if filename.endswith('.pdf'):
        filepath = os.path.join(resumes_dir, filename)
        resume_data = extract_text_from_pdf(filepath)
        data.append({'file_path': filepath, 'All': resume_data})
df2 = pd.DataFrame(data)

# Merge the two data frames on the "file_path" column
merged_df = pd.merge(df, df2, on='file_path')
merged_df.drop(columns=["college_name","company_names","designation","experience","total_experience"], axis=1, inplace=True) # dropping unwanted col

# Define the regex pattern to match unwanted characters
unwanted_chars_pattern = r'[^a-zA-Z !@#$%&*_+-=|\:";<>,./()[\]{}\']'
for col in ['skills','degree', 'All']:
    merged_df[col] = merged_df[col].apply(lambda x: re.sub(unwanted_chars_pattern, '', str(x)))


merged_df


In [ ]:
merged_df.columns

In [ ]:
# df2 = merged_df.copy()

In [ ]:
merged_df.to_csv('cv.csv')

In [ ]:
df=pd.read_csv("cv.csv")

In [ ]:
df.head()

In [ ]:
import os
import pandas as pd
from pyresparser import ResumeParser
import pdfplumber
import re

In [ ]:
def pdf_to_base64(file_path):
    #with pdfplumber.open(filepath) as pdf
    with open(file_path, 'rb') as f:
    
        encoded_pdf = base64.b64encode(f.read()).decode('utf-8')

    return encoded_pdf

In [ ]:
encoded_files=[]
for i in df["file_path"].values:
    print()
    encoded_files.append(pdf_to_base64(i))

In [ ]:
df["pdf_to_base64"]=encoded_files

In [ ]:
df.to_csv("updated_cv.csv",index=False)